In [1]:
#Sample caching code using python dictionary


import time
import json
import math

class Cache():

    def __init__(self):
        self.cache = {}
        self.idle_times = {}

    def data_modifier(self, data):
        res = {'res': data}
        json_res = json.dumps(res,ensure_ascii=False).encode('utf8')
        return json_res

    def clear_20_percent(self):
        print('CLEARING')
        # First sorting searches
        keys, idle_times = [], []
        for key in self.cache:
            keys.append(key)
            idle_times.append(time.time() - self.idle_times[key])

        keys = [k for i,k in sorted(zip(idle_times, keys))]
        target = math.ceil(len(keys)*0.2)

        # deleting oldest 20% of keys
        for i in range(target):
            self.cache.popitem()

        # if current cache memory still more than 10 mb [only a safety check]
        cur_mem = self.convert_to_num(str(self.cache.__sizeof__()))
        while(cur_mem > 10):
            self.cache.popitem()
            cur_mem = self.convert_to_num(str(self.cache.__sizeof__()))

    def push_to_dynamic_cache(self, key, data):
        data = self.data_modifier(data)
        if len(self.cache) < 10:
            self.cache[key] = data
            self.idle_times[key] = time.time()
        else:
            self.clear_20_percent()
            self.cache[key] = data
            self.idle_times[key] = time.time()

    def get_from_dynamic_cache(self, key):
        try:
            data = self.cache[key]
            self.idle_times[key] = time.time()
            return json.loads(data.decode())['res']
        except:
            return -1

    def push_to_top10_cache(self, key, data):
        data = self.data_modifier(data)
        self.cache[key] = data
        self.idle_times[key] = time.time()

    def get_from_top10_cache(self, key):
        try:
            data = self.cache[key]
            self.idle_times[key] = time.time()            
            return json.loads(data.decode())['res']
        except:
            return -1


In [6]:
# this code block allows you to inspect 2 tweets and 2 retweets in the dataset
# it also saves all the unique tweets/retweets in the dictionary tweets
# this code will help you to understand the data
# feel free to print out more fields

lines=0
cache={}

#insert path and replace name of the file below as needed
with open("corona-out-2.json", "r") as f1:
    for line in f1:
        try:
            data = json.loads(line)
            lines = lines + 1
            cache[data['id']]=data['user']['name']
            
        except:
            # if there is an error loading the json of the tweet, skip
            continue

In [7]:
cache

{1249403767180668930: 'juwelz v',
 1249403768023678982: 'efe09',
 1249403769193779202: 'Ravin Gupta',
 1249403769567227906: 'Carpe diem',
 1249403770435493888: 'UpsidedownTurtle 🧢',
 1249403770360016896: 'TV Conservador / Metró',
 1249403770833981440: 'Charles Mire',
 1249403771077005315: 'Ashu',
 1249403771261722624: 'Elfpunkt',
 1249403771676815361: 'Abhisht',
 1249403771781816323: 'Rizzu',
 1249403772679438340: 'Erin',
 1249403773178560515: 'Ruh-u Revan',
 1249403773476356096: 'Zekeriya_EYT_Şahin',
 1249403773522337792: 'Pritee Dhowtall',
 1249403773740552193: 'Carla Soares de Cast',
 1249403773799170049: 'Andra',
 1249403774151659521: 'Wiltac',
 1249403774583689221: 'Fred99 🇩🇪 🇮🇱',
 1249403775229534209: 'Yash Foundation',
 1249403775493816321: 'Rick James🦋',
 1249403775632076801: 'DavidGultomteknisilistrik',
 1249403775728717825: '🌹',
 1249403775812612096: 'fairy Tae ʚïɞ',
 1249403776441737216: 'magic',
 1249403776492015618: 'Lori Harvey',
 1249403777498542081: 'judy',
 12494037775

In [8]:
myCache = Cache()

In [9]:
# this code block allows you to inspect 2 tweets and 2 retweets in the dataset
# it also saves all the unique tweets/retweets in the dictionary tweets
# this code will help you to understand the data
# feel free to print out more fields

lines=0

#insert path and replace name of the file below as needed
with open("corona-out-2.json", "r") as f1:
    for line in f1:
        try:
            data = json.loads(line)
            lines = lines + 1
            myCache.push_to_dynamic_cache(data['id'], data['user']['name'])
            
        except:
            # if there is an error loading the json of the tweet, skip
            continue

CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
CLEARING
C